**Note that : To test the models with the test set you have ready in a file , just appload the file and change the name of the test file path in the last cell then run it to get the performance of the test**

In [4]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from scipy.signal import cwt, morlet, stft
import cv2
from PIL import Image

def evaluate_models_and_predict(test_path):
    # Load the test dataset
    test_data = np.loadtxt(test_path)

    # Separate signals and labels
    test_labels = test_data[:, 0]
    test_signals = test_data[:, 1:]

    # Convert labels to 0 and 1 if they are -1 and 1 respectively
    test_labels = np.where(test_labels == -1, 0, test_labels)

    print("Test dataset loaded and preprocessed.")

    # Dictionary to store models and their expected input shapes
    models = {
        'RP': ('cnn_rp_model.h5', (32, 32, 1)),
        'GADF': ('cnn_gadf_model.h5', (96, 96, 1)),
        'STFT': ('cnn_stft_model.h5', (25, 5, 1)),
        'CWT': ('cnn_cwt_model.h5', (96, 30, 1))
    }

    # Initialize dictionary to store classification reports
    classification_reports = {}

    # Function to preprocess test signals for RP model
    def preprocess_rp(signals):
        def paa(data, ds_factor=2):
            d, ds_b = data.shape
            ds_length = int(ds_b / ds_factor)
            ds_series = np.empty((d, ds_length))
            for i in range(ds_length):
                ds_series[:, i] = np.mean(data[:, i*ds_factor:(i+1)*ds_factor], axis=1)
            return ds_series
        
        def r_plot(data, delay=0):
            transformed = np.zeros([2, len(data) - delay])
            transformed[0, :] = data[0:len(data) - delay]
            transformed[1, :] = data[delay:len(data)]
            rp = np.zeros([len(data) - delay, len(data) - delay])
            for i in range(len(rp)):
                temp = np.tile(transformed[:, i], (len(rp), 1)).T - transformed
                temp2 = np.square(temp)
                rp[i, :] = np.sum(temp2, axis=0)
            return np.array(rp)

        ds_signals = paa(signals, ds_factor=3)
        rp_images = np.empty((len(ds_signals), 32, 32))
        for i in range(len(ds_signals)):
            rp_images[i, :, :] = r_plot(ds_signals[i, :])
        rp_images = rp_images[..., np.newaxis]  # Add channel dimension
        return rp_images

    # Function to preprocess test signals for GADF model
    def preprocess_gadf(signals):
        def rescale(data):
            num = data - np.tile(np.mat(data.min(axis=1)).T, (1, np.shape(data)[1]))
            denom = np.tile(np.mat(data.max(axis=1)).T, (1, np.shape(data)[1])) - np.tile(np.mat(data.min(axis=1)).T, (1, np.shape(data)[1]))
            rescaled = np.multiply(num, 1 / denom)
            return rescaled

        def GADF(data):
            datacos = np.array(data)
            datasin = np.sqrt(1 - datacos**2)
            gadf = datasin.T * datacos - datacos.T * datasin
            return gadf
        
        rescaled_signals = rescale(signals)
        gadf_images = np.empty((len(rescaled_signals), 96, 96))
        for i in range(len(rescaled_signals)):
            gadf_image = GADF(rescaled_signals[i, :])
            gadf_image_resized = Image.fromarray(gadf_image).resize((96, 96))  # Resize to (96, 96) using PIL
            gadf_images[i, :, :] = np.array(gadf_image_resized)
        gadf_images = gadf_images[..., np.newaxis]  # Add channel dimension
        return gadf_images

    # Function to preprocess test signals for STFT model
    def preprocess_stft(signals):
        def compute_stft(signal, fs=100, nperseg=32, noverlap=16):
            _, _, Zxx = stft(signal, fs=fs, nperseg=nperseg, noverlap=noverlap)
            return np.abs(Zxx)
        
        expected_size = (25, 5)
        stft_images = np.empty((len(signals), *expected_size))
        for i in range(len(signals)):
            stft_image = compute_stft(signals[i, :])
            stft_image_resized = cv2.resize(stft_image, (expected_size[1], expected_size[0]))
            stft_images[i, :, :] = stft_image_resized
        stft_images = stft_images[..., np.newaxis]  # Add channel dimension
        return stft_images

    # Function to preprocess test signals for CWT model
    def preprocess_cwt(signals):
        def compute_cwt(signal, widths=np.arange(1, 31)):
            cwt_matr = cwt(signal, morlet, widths)
            return np.abs(cwt_matr)
        
        expected_size = (96, 30)
        cwt_images = np.empty((len(signals), *expected_size))
        for i in range(len(signals)):
            cwt_image = compute_cwt(signals[i, :])
            cwt_image_resized = cv2.resize(cwt_image, (expected_size[1], expected_size[0]))
            cwt_images[i, :, :] = cwt_image_resized
        cwt_images = cwt_images[..., np.newaxis]  # Add channel dimension
        return cwt_images

    # Mapping model names to preprocessing functions
    preprocessing_functions = {
        'RP': preprocess_rp,
        'GADF': preprocess_gadf,
        'STFT': preprocess_stft,
        'CWT': preprocess_cwt
    }

    # Evaluate each model and store the best one
    best_model = None
    best_accuracy = 0.0
    best_predictions = None

    for model_name, (model_file, expected_input_shape) in models.items():
        print(f"Evaluating model: {model_name}")
        
        # Load model
        model = load_model(model_file)
        
        # Preprocess test data
        preprocessed_test_images = preprocessing_functions[model_name](test_signals)
        
        # Ensure the shape of the test images matches the model's expected input size
        if preprocessed_test_images.shape[1:] != expected_input_shape:
            print(f"Resizing images for {model_name} to match the model's expected input shape.")
            resized_images = np.empty((len(preprocessed_test_images), *expected_input_shape[:-1]))
            for i in range(len(preprocessed_test_images)):
                resized_image = cv2.resize(preprocessed_test_images[i, :, :, 0], (expected_input_shape[1], expected_input_shape[0]))
                resized_images[i, :, :] = resized_image
            preprocessed_test_images = resized_images[..., np.newaxis]  # Add channel dimension

        # Make predictions
        y_pred = np.argmax(model.predict(preprocessed_test_images), axis=1)
        
        # Print and store the classification report
        report = classification_report(test_labels, y_pred, output_dict=True)
        classification_reports[model_name] = report
        print(f"Classification Report for {model_name} Model on Test Data:")
        print(classification_report(test_labels, y_pred))

        # Check if this model is the best so far
        accuracy = report['accuracy']
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model_name
            best_predictions = y_pred

    print(f"The best model is {best_model} with an accuracy of {best_accuracy:.2f}")

    return best_predictions




In [ ]:
# Example usage
test_dataset_path = #'ECG200_TEST.txt'  # Replace with the actual path to the test dataset 
predictions = evaluate_models_and_predict(test_dataset_path)
print(f"Predicted classifications using the best model: {predictions}")